In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# lit le fichier
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Data Cleaning
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# on separe les données validation et test
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

#colonnes ayant une cardinalité faible pour la simplicité
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# colonnes d'entiers
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# on garde seulement ces colonnes
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# on utilise one hot encode
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

Essayons **trois** differents maniere de booster le modele 

**Premier essai**
* Utilisons xgboost sans regler des parametres

In [2]:
from xgboost import XGBRegressor
my_model_1 = XGBRegressor()
my_model_1.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [3]:
from sklearn.metrics import mean_absolute_error
predictions_1 = my_model_1.predict(X_valid)

In [4]:
mae_1 = mean_absolute_error(predictions_1, y_valid)
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 17662.736729452055


le resultat n'est pas aussi bas que desiré

**Deuxieme essai**
* Utilisons 1000 cycles, les resultats de chaque cycle seront ajouté
* avec un multiplicateur de 0.05

In [5]:
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model_2.fit(X_train, y_train)

# estimations
predictions_2 = my_model_2.predict(X_valid)

# MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)

print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 16688.691513270547


C'est effectivement mieux

**Troiseme essai**
* essayons avec qu'une cycle

In [6]:
# defini le modele
my_model_3 = XGBRegressor(n_estimators=1)
#

# adapte le modele
my_model_3.fit(X_train, y_train)

predictions_3 = my_model_3.predict(X_valid)

# calcul MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)

print("Mean Absolute Error:" , mae_3)


Mean Absolute Error: 127895.0828807256


* sans surprise ça marche pas tres bien

Notre meilleur modele est donc le deuxieme 

In [7]:
# on fait la prediction sur les données test
preds_test = my_model_2.predict(X_test)

In [8]:
# on stocke dans un fichier csv
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)
print("terminé")

terminé
